# Part 1: Data Cleaning
----

# To TAs/Graders:

### The following below is so important and you could not want to miss.

This `data_cleaning.ipynb` file is tedious and boring, which not only

- **needs domain knowledge**, but also 
- **to re-run some code**.

It is very suggested to go through this file, and to use the `data/data_com.csv` for other .ipynb.

### Explanation for `temp` directory.

It is like a playground.

No need to worry the important data and models used in other notebook will be re-write, appended, or changed by some running error.

- The **raw data** are saved in `raw_data` directory.

- The **cleaned data** are saved in `data` directory, which will be used by other .ipynb files.

- The **dataset created in this notebook** are saved in `temp/data` directory. Because we assumed you run this code only for checking the code, It is a temp directory, saving temporary data.

----

# Note
This file is used to proprocess the fiscal data and graduation data of year 2007 to 2010. Data was cleaned, selectd, merged between table, and combined from different years. The processed data has one identification column 'LEAID', one target column 'AFGR' representing graduation rate, and other feature columns.

# Table of contents
1. Import data
2. Data cleaning
3. Combine variables based on explanation from the data manual
4. Output
5. Combine four data files

# 1. Import data
---
Import both fiscal and graduation data

## a. Look at data

In [1]:
import pandas as pd

In [2]:
# the following code is for data from in year 2008. 
# For other years, change the file name, eg. 'data/fiscal08-09.txt' and 'data/dr08-09.txt' for year 2009
fiscal_path = 'raw_data/fiscal07-08.txt'
dropout_path = 'raw_data/dr07-08.txt'

In [3]:
def import_data(path):
    imported_data = pd.read_csv(path, sep='\t', low_memory=False, dtype = {'LEAID':str})
    return imported_data
#dtype={"LEAID": str,'SCHLEV':str, 'GSLO':str}

In [4]:
fiscal = import_data(fiscal_path)

In [5]:
# only year 2008 has graduation data in which the 'leaid' is in lowercase
dropout = pd.read_csv(dropout_path, sep='\t', dtype = {'leaid':str})

# for other years run the following code
# dropout = pd.read_csv(dropout_path, sep='\t', dtype = {'LEAID':str})

In [6]:
# If no dtype is not specified, there will be an error.
# DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.

In [7]:
fiscal.head()

,LEAID,CENSUSID,FIPST,CONUM,CSA,CBSA,NAME,STNAME,STABBR,SCHLEV,...,FL_V93,FL_19H,FL_21F,FL_31F,FL_41F,FL_61V,FL_66V,FL_W01,FL_W31,FL_W61
0,0100005,01504840100000,1,1095,N,10700,ALBERTVILLE CITY SCHOOL DISTRICT,Alabama,AL,3,...,R,R,R,R,R,R,R,R,R,R
1,0100006,01504800100000,1,1095,N,10700,MARSHALL COUNTY SCHOOL DISTRICT,Alabama,AL,3,...,R,R,R,R,R,R,R,R,R,R
2,0100007,01503740100000,1,1073,142,13820,HOOVER CITY SCHOOL DISTRICT,Alabama,AL,3,...,R,R,R,R,R,R,R,R,R,R
3,0100008,01504530100000,1,1089,290,26620,MADISON CITY SCHOOL DISTRICT,Alabama,AL,3,...,R,R,R,R,R,R,R,R,R,R
4,0100011,01503710100000,1,1073,142,13820,LEEDS CITY SCHOOL DISTRICT,Alabama,AL,3,...,R,R,R,R,R,R,R,R,R,R


In [1]:
#fiscal.info()

In [9]:
dropout.head()

,survyear,fipst,leaid,totd912,ebs912,drp912,totdpl,afgeb,afgr,totohc
0,2007-08,1,0100002,0,47,0.0,-1,14,-1.0,-1
1,2007-08,1,0100005,29,939,3.1,172,247,69.6,5
2,2007-08,1,0100006,41,1612,2.5,276,450,61.3,13
3,2007-08,1,0100007,38,3817,1.0,899,978,91.9,5
4,2007-08,1,0100008,27,2715,1.0,624,584,100.0,9


In [2]:
#dropout.info()

## b. Check duplicated data for each dataframe

In [11]:
len(fiscal['LEAID'])

16453

In [12]:
len(fiscal[fiscal['LEAID'].duplicated() == True])

0

In [13]:
len(dropout['leaid'])

18090

In [14]:
len(dropout[dropout['leaid'].duplicated() == True])

0

In [15]:
# only run this for year 2008 becuase the labels is in lowercase
dropout['LEAID'] = dropout['leaid']
dropout['AFGR'] = dropout['afgr']

## c. Check number of same LEAID for both dataframe
LEAID is used to join the fiscal table and the graduation table

In [16]:
len(set([i for i in fiscal['LEAID']]) & set([i for i in dropout['LEAID']]))

16408

# 2. Data cleaning
---

## a. Selection variables that are going to be used

In [17]:
# ignore columns that are flags for the numerical variables
# for year 2007， there is one more data column 'C18'
fiscal = fiscal[['LEAID','SCHLEV','AGCHRT','CONUM','FIPST','YEAR','V33',
                 'C14','C16','C17','C25',
                 'C15','C18','C19','B11','B10','B12',
                 'C20','C36','B13',
                 'C01','C04','C10','C12','C38',
                 'C05','C06','C07','C08','C09',
                 'C11','C13','C35','C39',
                 'T06','T09','T15','T40','T99','D11','D23',
                 'A07','A08','A09','A11','A13','A15','A20','A40',
                 'U11','U22','U30','U50','U97','C24',
                 'Z33',
                 'V11','V13','V15','V17','V21','V23','V37','V29',
                 'Z34',
                 'E13','TCURSSVC','E11','V60','V65',
                 'TNONELSE',
                 'TCAPOUT',
                 'L12','M12','Q11','V91','V92',
                 'V93',
                 'I86'                 
                ]]

In [3]:
#fiscal.info()

In [19]:
# There are no null value for both dataset.

## b. Deal with invalid value

#### 1) almost every column has "-2, -1, 0",

Take column "V93" for Example:

-2 5.935222

-1 3.490985

0 32.690305

We don't know the meaning of -1 and -2, and difference between -1 and -2

Almost every column's "-2, -1" has the same proportion, 5.935222 and 3.490985

-2 5.935222

-1 3.490985

**We asume that, this should be a series of effects which means drop rows where value = -1 or -2 for one column, could make every column clean.**

But 0 means 0. There is no need to delete 0.

In [4]:
#for a in fiscal.columns:
#    l = len(fiscal)
#    print(fiscal.groupby(by=[a])[a].count()/l*100)

#### 2) Missing, Nonapplicable, and Suppressed Data (interpretion for negative values)

-1 In the F-33 data files, CCD identifies missing data by reporting the data value as “-1.”

-3 CCD identifies suppressed membership data by reporting the membership as “-3” and the membership flag as a value of “A.”

-9 CCD identifies submitted F-33 data that do not meet NCES data quality standards by reporting the data item as “-9” and data item flag as “A.”

-2 CCD identifies nonapplicable data by reporting the data value as “-2” and the data item flag as a value of “N.”

In [21]:
# get rid of all negative data in columns that they appear
for a in ['T06','T09','T15','T40','T99','V33']:
    fiscal = fiscal[fiscal[a] >= 0 ]

Check if -1 or -2 still in one of the columns

In [5]:
#for a in fiscal.columns:
#    l = len(fiscal)
#    print(fiscal.groupby(by=[a])[a].count()/l*100)

In [23]:
print(len(fiscal))

12777


In [6]:
# check which column has null value
#fiscal.isnull().any()

# fiscal.dropna(how = 'any',axis = 0)
# fiscal.dropna(how = 'any',axis = 1)

##### AGCHRT: 

AGENCY CHARTER CODE

1 = All schools are charter schools

2 = All schools are charter and noncharter schools

3 = All associated schools are noncharter schools

N = Not applicable or code could not be determined

**Just treat them as categorical data, no need to change.**

### C.Deal with graduation data

In [8]:
#for a in dropout.columns:
#   l = len(dropout)
#    print(dropout.groupby(by=[a])[a].count()/l*100)

In [9]:
#dropout.head(5)

#### 1) Dropout Data - Missing, Nonapplicable, and Suppressed Data from the data manual

Data suppression has also been employed as part of the CCD disclosure mitigation plan. Dropout
counts of 1, 2, or 3 have been suppressed. These counts are presented on the data file with the
value -3. Dropout counts that are 1, 2, or 3 students less than the membership count have also
been suppressed. These counts are represented on the file with the denoted value of -4. In order
to prevent data users from backing out these suppressed values and determining the real value of
the cell, complimentary suppression has also been employed. Any complementary suppression
performed on the file is denoted with the same value as a missing count, -1. These suppressed
cells are not distinguishable from the cells that contain missing values.

Suppression has also been employed to protect against the individual disclosure of anyone that
did not receive a regular high school diploma following their 12th grade year. These, and the
counter-suppressions made to protect the primary suppressions, are denoted as -1 on the data file.
These suppressed cells are not distinguishable from cells that contain missing values. 

0: A zero value represents a report of no occurrences of a data element. A value was
expected and measured, but zero cases were found in the category. (For example, a K–12
district having no 12th-graders would report “0.”) 

M (or -1 for numeric values): A value of M (or -1) indicates that data are missing. A
value was expected, but none was measured. (For example, a district that has at least one
12th-grader but cannot measure the number of 12th-graders would report “-1.”). This value
also denotes a suppressed high school diploma count or dropout count. 

N (or -2 for numeric values): A value of N (or -2) indicates that data are not applicable. A
value was neither expected nor measured. (For example, an elementary school district
would report “-2” for 12th-graders.)

-3: A value of -3 indicates a dropout count of 1, 2, or 3. These cells have been suppressed
such that the true value of the cell cannot be identified. All cells with a value of -3 have a
plausible value of 1, 2, or 3. 

-4: A value of -4 indicates a dropout count that is equal to or exceeds the 3 less than the
membership count. These cells have been suppressed such that the true value of the cell
cannot be identified. All cells with a value of -4 have a plausible value of 3 less than the
membership. 

#### 2) Look at the graduation rate column

AFGR is more clean than DRP912, there only 3 negative values.

**For AFGR:**

-9.0       1.090081

-2.0      35.066978

-1.0       5.314822

**For DRP912:**
    
-9.0      4.051196

-4.0      2.717067

-3.0     50.111177

-2.0     28.472260

-1.0      2.467596

**So We are tring to use AFGR. This will remove less instances.**

In [27]:
# remove all negative numbers
dropout = dropout[['LEAID','AFGR']]
dropout = dropout[dropout['AFGR'] != -9]
dropout = dropout[dropout['AFGR'] != -2]
dropout = dropout[dropout['AFGR'] != -1]

In [28]:
len(dropout)

10735

In [10]:
#for a in dropout.columns:
#    l = len(dropout)
#    print(dropout.groupby(by=[a])[a].count()/l*100)

We've got a clean AFGR column, just forget the others now.

In [11]:
#dropout.info()

In [12]:
#fiscal.info()

### D. Merge fiscal and graduation table based on 'LEAID'

In [32]:
data = pd.merge(fiscal, dropout, how='inner', on="LEAID", suffixes=('_f', '_d'))

In [33]:
data.head()

,LEAID,SCHLEV,AGCHRT,CONUM,FIPST,YEAR,V33,C14,C16,C17,...,TNONELSE,TCAPOUT,L12,M12,Q11,V91,V92,V93,I86,AFGR
0,0100005,3,3,1095,1,8,3790,1053000,163000,65000,...,736000,12082000,0,0,70000,0,0,240000,413000,69.6
1,0100006,3,3,1095,1,8,5647,2288000,424000,558000,...,1491000,2443000,0,0,8000,0,0,555000,746000,61.3
2,0100007,3,3,1073,1,8,12479,165000,165000,21000,...,4709000,5784000,0,0,99000,0,0,1876000,9415000,91.9
3,0100008,3,3,1089,1,8,8298,589000,155000,16000,...,1245000,8347000,0,0,124000,8000,0,798000,3316000,100.0
4,0100011,3,3,1073,1,8,1406,344000,100000,6000,...,138000,1575000,0,0,13000,0,0,84000,182000,67.3


In [34]:
len(data)

9527

In [13]:
# look at the 
#data.info()

In [14]:
# Look at data distribution
#for a in data.columns:
#    l = len(data)
#    print(data.groupby(by=[a])[a].count()/l*100)

# 3. Combine variables based on explanation from the data manual
---

In [37]:
data.head()

,LEAID,SCHLEV,AGCHRT,CONUM,FIPST,YEAR,V33,C14,C16,C17,...,TNONELSE,TCAPOUT,L12,M12,Q11,V91,V92,V93,I86,AFGR
0,0100005,3,3,1095,1,8,3790,1053000,163000,65000,...,736000,12082000,0,0,70000,0,0,240000,413000,69.6
1,0100006,3,3,1095,1,8,5647,2288000,424000,558000,...,1491000,2443000,0,0,8000,0,0,555000,746000,61.3
2,0100007,3,3,1073,1,8,12479,165000,165000,21000,...,4709000,5784000,0,0,99000,0,0,1876000,9415000,91.9
3,0100008,3,3,1089,1,8,8298,589000,155000,16000,...,1245000,8347000,0,0,124000,8000,0,798000,3316000,100.0
4,0100011,3,3,1073,1,8,1406,344000,100000,6000,...,138000,1575000,0,0,13000,0,0,84000,182000,67.3


In [38]:
# One more column 'C18' for year 2007, should be grouped into ‘Re_F_Special’
data['Re_F_Basic'] = data['C14'] + data['C16'] + data['C17'] + data['C25']
data['Re_F_Special'] = data['C15'] + data['C19'] + data['B11'] + data['B10'] + data['B12']
data['Re_F_Other'] = data['C20'] + data['C36'] + data['B13']
data['Re_S_Basic'] = data['C01'] + data['C04'] + data['C10'] + data['C12'] + data['C38']
data['Re_S_Special'] = data['C05'] + data['C06'] + data['C07'] + data['C08'] + data['C09']
data['Re_S_Other'] = data['C11'] + data['C13'] + data['C35'] + data['C39']
data['Re_L_Gov'] = data['T06'] + data['T09'] + data['T15'] + data['T40'] + data['T99'] + data['D11'] + data['D23']
data['Re_L_fee'] = data['A07'] + data['A08'] + data['A09'] + data['A11'] + data['A13'] + data['A15'] + data['A20'] + data['A40']
data['Re_L_Other'] = data['U11'] + data['U22'] + data['U30'] + data['U50'] + data['U97'] + data['C24']
data['Ex_Teacher_Inst'] = data['Z33']
data['Ex_Teacher_Supp'] = data['V11'] + data['V13'] + data['V15'] + data['V17'] + data['V21'] + data['V23'] + data['V37'] + data['V29'] 
data['Ex_Employ'] = data['Z34']
data['Ex_Edu'] = data['E13'] + data['TCURSSVC'] + data['E11'] + data['V60'] + data['V65']
data['Ex_Community'] = data['TNONELSE']
data['Ex_Capital'] = data['TCAPOUT']
data['Ex_Payment'] = data['L12'] + data['M12'] + data['Q11'] + data['V91'] + data['V92']
data['Ex_Textbook'] = data['V93']
data['Ex_Interest'] = data['I86']

In [39]:
data = data[['LEAID','SCHLEV','AGCHRT','CONUM','FIPST','YEAR','V33',
             'Re_F_Basic','Re_F_Special','Re_F_Other',
             'Re_S_Basic','Re_S_Special','Re_S_Other',
             'Re_L_Gov','Re_L_fee','Re_L_Other',
             'Ex_Teacher_Inst','Ex_Teacher_Supp','Ex_Employ',
             'Ex_Edu','Ex_Community','Ex_Capital','Ex_Payment',
             'Ex_Textbook','Ex_Interest',
             'AFGR'
            ]]

In [15]:
#data.info()

In [41]:
data.head()

,LEAID,SCHLEV,AGCHRT,CONUM,FIPST,YEAR,V33,Re_F_Basic,Re_F_Special,Re_F_Other,...,Ex_Teacher_Inst,Ex_Teacher_Supp,Ex_Employ,Ex_Edu,Ex_Community,Ex_Capital,Ex_Payment,Ex_Textbook,Ex_Interest,AFGR
0,0100005,3,3,1095,1,8,3790,2482000,1101000,224000,...,12471000,5847000,7745000,32292000,736000,12082000,70000,240000,413000,69.6
1,0100006,3,3,1095,1,8,5647,5309000,1408000,242000,...,17798000,10404000,11933000,50516000,1491000,2443000,8000,555000,746000,61.3
2,0100007,3,3,1073,1,8,12479,1513000,2158000,159000,...,52946000,28252000,34750000,148790000,4709000,5784000,99000,1876000,9415000,91.9
3,0100008,3,3,1089,1,8,8298,1582000,1738000,46000,...,27664000,12089000,16632000,72287000,1245000,8347000,132000,798000,3316000,100.0
4,0100011,3,3,1073,1,8,1406,839000,346000,0,...,4262000,2685000,2990000,12542000,138000,1575000,13000,84000,182000,67.3


# 4. Output
---
To get cleaned and merged data in each year, the code needs to be rerun again with changes noted in the code.

##### Because we assumed you run this code only for checking the code, we put the `07-08data.csv` in a temp directory, saving temporary data.

In [42]:
# save the data to the correspoding csv file for the year
#data.to_csv('06-07data.csv')
data.to_csv('temp/data/07-08data.csv')
#data.to_csv('08-09data.csv')
#data.to_csv('09-10data.csv')

In [43]:
#data0607 = data
data0708 = data
#data0809 = data
#data0910 = data

# 5. Combine four data files
---
This can only be run when data files from four different years are all created.

In [44]:
#data0607.info()
#data0708.info()
#data0809.info()
#data0910.info()

In [45]:
# combine files from the four years
# data_com = pd.concat([data0607,data0708,data0809,data0910])
# data_com.info()

In [46]:
# # save the complete csv file
# data_com.to_csv('data_com.csv')

The Final data file `data_com.csv` is generated by this code above, which is commented.
While, to generate this, needs all the files available. You can re-run the code and follow the comments to slightly change some line of this code.

##### But all the procedure in other ipynb are using the dataset in `data/data_com.csv`, which is the final dataset we have already created.